In [1]:
import numpy as np
import cv2
import os
from os import listdir
from os.path import isfile, isdir, join
from matplotlib import pyplot as plt

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.layers import Input, Dense, Conv2D, Flatten
from keras.layers.merge import concatenate
from keras import Model

Using plaidml.keras.backend backend.


In [2]:
class DescriptorNet(keras.Model):
    def __init__(self):
        super(DescriptorNet, self).__init__()
        # Input 1 ground truth descriptors
        self.convA1 = Conv2D(1000, kernel_size=1, activation='relu')
        self.convA2 = Conv2D(500, kernel_size=1, activation='relu')
        self.convA3 = Conv2D(250, kernel_size=1, activation='relu')
        self.convA4 = Conv2D(125, kernel_size=1, activation='relu')
        self.convA5 = Conv2D(64, kernel_size=1, activation='relu')
        self.convA6 = Conv2D(32, kernel_size=1, activation='relu')
        self.convA7 = Conv2D(16, kernel_size=1, activation='relu')
        self.flatten1 = Flatten()
        
        # Input 2, NIR descriptors
        self.convB1 = Conv2D(1000, kernel_size=1, activation='relu')
        self.convB2 = Conv2D(500, kernel_size=1, activation='relu')
        self.convB3 = Conv2D(250, kernel_size=1, activation='relu')
        self.convB4 = Conv2D(125, kernel_size=1, activation='relu')
        self.convB5 = Conv2D(64, kernel_size=1, activation='relu')
        self.convB6 = Conv2D(32, kernel_size=1, activation='relu')
        self.convB7 = Conv2D(16, kernel_size=1, activation='relu')
        self.flatten2 = Flatten()
        
        # Merge
        self.dense1 = Dense(2048 , activation='relu')
        self.dense2 = Dense(1024 , activation='relu')
        self.outputs = Dense(8 , activation='linear')
        
    def call(self, inputs):
        x = inputs[0]
        
        x = self.convA1(x)
        x = self.convA2(x)
        x = self.convA3(x)
        x = self.convA4(x)
        x = self.convA5(x)
        x = self.convA6(x)
        x = self.convA7(x)
        x = self.flatten1(x)
        
        y = inputs[1]
        y = self.convB1(y)
        y = self.convB2(y)
        y = self.convB3(y)
        y = self.convB4(y)
        y = self.convB5(y)
        y = self.convB6(y)
        y = self.convB7(y)
        y = self.flatten2(y)
        
        z = concatenate([x, y])
        
        z = self.dense1(z)
        z = self.dense2(z)
        
        return self.outputs(z)
    
    def model(self):
        x = Input(shape=(1, 128, 2000))
        y = Input(shape=(1, 128, 2000))
        return Model(inputs=[x, y], outputs=self.call([x, y]))
        

In [3]:
DescriptorNet().model().summary()

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 128, 2000) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 128, 2000) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1, 128, 1000) 2001000     input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 1, 128, 1000) 2001000     input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (